# PHASE 2: EXPLORATORY DATA ANALYSIS

## Objectives
This phase conducts deep analysis across six key dimensions:

1. **Temporal Analysis** - Demand patterns, peak hours, seasonal trends
2. **Geographic Analysis** - Zone-level activity, transport deserts, hotspots
3. **Service Equity** - Wait time disparities, underserved areas, accessibility gaps
4. **Uber vs Lyft Comparison** - Market dynamics, pricing, service quality
5. **Environmental Impact** - Emissions by zone, pollution hotspots, carbon intensity
6. **Economic Analysis** - Driver earnings, fare structures, income distribution

In [0]:
# Loading the cleaned data from Delta Lake that I saved in Phase 1

print("Loading cleaned dataset...")

df = spark.table("nyc_ridehail_q1_2025_clean")

print(f"✅ Data loaded: {df.count():,} records")
print(f"Columns: {len(df.columns)}")

# Quick verification that all my calculated fields are present
required_cols = ['company_name', 'pickup_hour', 'day_name', 'wait_time_minutes', 
                 'fare_per_mile', 'driver_pct', 'passenger_total_paid', 'co2_kg', 'speed_mph']

missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"⚠️ Missing columns: {missing_cols}")
else:
    print("✅ All required columns present")

In [0]:
# I'm importing all the tools I need for data processing and visualization
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np

# Setting up my visualization preferences
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries loaded successfully")
print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [0]:
# Looking at how ride patterns changed across the three months of Q1

print("Analyzing monthly trends...")

monthly_stats = df.groupBy('pickup_month').agg(
    F.count('*').alias('total_trips'),
    F.avg('wait_time_minutes').alias('avg_wait'),
    F.avg('base_passenger_fare').alias('avg_fare'),
    F.avg('trip_miles').alias('avg_distance'),
    F.sum('co2_kg').alias('total_co2_kg')
).orderBy('pickup_month').toPandas()

# Adding month names for better readability
month_names = {1: 'January', 2: 'February', 3: 'March'}
monthly_stats['month_name'] = monthly_stats['pickup_month'].map(month_names)

print("\nMonthly Statistics:")
print(monthly_stats)

# Creating a comprehensive monthly comparison chart
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Panel 1: Total trips per month
axes[0, 0].bar(monthly_stats['month_name'], monthly_stats['total_trips'], 
               color=['#3498db', '#e74c3c', '#2ecc71'], edgecolor='black', linewidth=1.5)
axes[0, 0].set_ylabel('Number of Trips', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Trip Volume by Month', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(monthly_stats['total_trips']):
    axes[0, 0].text(i, v + 100000, f"{v/1e6:.1f}M", ha='center', fontweight='bold')

# Panel 2: Average wait times
axes[0, 1].plot(monthly_stats['month_name'], monthly_stats['avg_wait'], 
                marker='o', linewidth=3, markersize=10, color='coral')
axes[0, 1].set_ylabel('Wait Time (minutes)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Average Wait Time by Month', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
for i, v in enumerate(monthly_stats['avg_wait']):
    axes[0, 1].text(i, v + 0.05, f"{v:.2f}", ha='center', fontweight='bold')

# Panel 3: Average fares
axes[1, 0].bar(monthly_stats['month_name'], monthly_stats['avg_fare'],
               color=['#9b59b6', '#f39c12', '#1abc9c'], edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('Average Fare ($)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Average Fare by Month', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(monthly_stats['avg_fare']):
    axes[1, 0].text(i, v + 0.2, f"${v:.2f}", ha='center', fontweight='bold')

# Panel 4: Total CO2 emissions
axes[1, 1].bar(monthly_stats['month_name'], monthly_stats['total_co2_kg']/1000,
               color=['#e67e22', '#16a085', '#c0392b'], edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('CO₂ Emissions (metric tons)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Monthly Carbon Emissions', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(monthly_stats['total_co2_kg']/1000):
    axes[1, 1].text(i, v + 200, f"{v/1000:.1f}K", ha='center', fontweight='bold')

plt.suptitle('Q1 2025 Monthly Trends Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/02_temporal/monthly_trends.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()


print("✅ Saved: monthly_trends.png")

In [0]:
# Comparing weekday and weekend patterns in detail

print("Analyzing weekday vs weekend patterns...")

weekend_comparison = df.groupBy('is_weekend').agg(
    F.count('*').alias('total_trips'),
    F.avg('wait_time_minutes').alias('avg_wait'),
    F.avg('base_passenger_fare').alias('avg_fare'),
    F.avg('tips').alias('avg_tips'),
    F.avg('driver_pay').alias('avg_driver_pay'),
    F.avg('trip_miles').alias('avg_distance'),
    F.avg('speed_mph').alias('avg_speed')
).toPandas()

# Adding readable labels
weekend_comparison['period'] = weekend_comparison['is_weekend'].map({'Y': 'Weekend', 'N': 'Weekday'})

print("\nWeekday vs Weekend Comparison:")
print(weekend_comparison)

# Calculating percentage differences
weekday_row = weekend_comparison[weekend_comparison['is_weekend'] == 'N'].iloc[0]
weekend_row = weekend_comparison[weekend_comparison['is_weekend'] == 'Y'].iloc[0]

print("\n📊 KEY DIFFERENCES (Weekend vs Weekday):")
print(f"  Trip Volume: {((weekend_row['total_trips'] - weekday_row['total_trips'])/weekday_row['total_trips']*100):+.1f}%")
print(f"  Wait Time: {((weekend_row['avg_wait'] - weekday_row['avg_wait'])/weekday_row['avg_wait']*100):+.1f}%")
print(f"  Fare: {((weekend_row['avg_fare'] - weekday_row['avg_fare'])/weekday_row['avg_fare']*100):+.1f}%")
print(f"  Tips: {((weekend_row['avg_tips'] - weekday_row['avg_tips'])/weekday_row['avg_tips']*100):+.1f}%")

# Creating comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

metrics = [
    ('total_trips', 'Total Trips', 'millions'),
    ('avg_wait', 'Avg Wait Time (min)', 'minutes'),
    ('avg_fare', 'Avg Fare ($)', 'dollars'),
    ('avg_tips', 'Avg Tips ($)', 'dollars'),
    ('avg_driver_pay', 'Avg Driver Pay ($)', 'dollars'),
    ('avg_speed', 'Avg Speed (mph)', 'mph')
]

for idx, (metric, title, unit) in enumerate(metrics):
    row = idx // 3
    col = idx % 3
    
    values = weekend_comparison[metric].values
    labels = weekend_comparison['period'].values
    colors = ['#3498db', '#e74c3c']
    
    bars = axes[row, col].bar(labels, values, color=colors, edgecolor='black', linewidth=1.5)
    axes[row, col].set_title(title, fontsize=12, fontweight='bold')
    axes[row, col].grid(True, alpha=0.3, axis='y')
    
    # Adding value labels on bars
    for i, (bar, val) in enumerate(zip(bars, values)):
        if metric == 'total_trips':
            label = f"{val/1e6:.1f}M"
        else:
            label = f"{val:.2f}"
        axes[row, col].text(bar.get_x() + bar.get_width()/2, val + max(values)*0.02, 
                           label, ha='center', fontweight='bold', fontsize=11)

plt.suptitle('Weekday vs Weekend Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/02_temporal/weekday_vs_weekend.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: weekday_vs_weekend.png")

In [0]:
# Identifying hours with lowest service availability (transport deserts)

print("Analyzing night transport deserts...")

# Defining night hours (midnight to 6 AM)
night_hours = list(range(0, 6))

hourly_volume = df.groupBy('pickup_hour').agg(
    F.count('*').alias('trip_count'),
    F.avg('wait_time_minutes').alias('avg_wait')
).orderBy('pickup_hour').toPandas()

# Calculating what percentage of daily trips happen each hour
hourly_volume['pct_of_daily'] = (hourly_volume['trip_count'] / hourly_volume['trip_count'].sum()) * 100

# Identifying the transport desert hours
hourly_volume['is_desert'] = hourly_volume['pickup_hour'].isin(night_hours)

print("\nNight Hours (12 AM - 6 AM) Statistics:")
night_data = hourly_volume[hourly_volume['is_desert']]
print(night_data[['pickup_hour', 'trip_count', 'pct_of_daily', 'avg_wait']])

total_night_trips = night_data['trip_count'].sum()
total_trips = hourly_volume['trip_count'].sum()
night_pct = (total_night_trips / total_trips) * 100

print(f"\n🌙 NIGHT TRANSPORT DESERT FINDINGS:")
print(f"  Night trips (12 AM - 6 AM): {total_night_trips:,} ({night_pct:.1f}% of daily volume)")
print(f"  Worst hour: {night_data.loc[night_data['trip_count'].idxmin(), 'pickup_hour']}:00 with {night_data['trip_count'].min():,} trips")
print(f"  Average night wait: {night_data['avg_wait'].mean():.2f} minutes")

# Creating visualization highlighting transport deserts
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Panel 1: Trip volume with desert hours highlighted
colors = ['#e74c3c' if h in night_hours else '#3498db' for h in hourly_volume['pickup_hour']]
axes[0].bar(hourly_volume['pickup_hour'], hourly_volume['trip_count'], 
           color=colors, edgecolor='black', linewidth=1)
axes[0].axvspan(-0.5, 5.5, alpha=0.2, color='red', label='Transport Desert (12 AM - 6 AM)')
axes[0].set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Trips', fontsize=12, fontweight='bold')
axes[0].set_title('Hourly Trip Volume - Transport Deserts Highlighted', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(0, 24))
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Panel 2: Percentage of daily volume
axes[1].plot(hourly_volume['pickup_hour'], hourly_volume['pct_of_daily'], 
            marker='o', linewidth=2.5, markersize=8, color='#2ecc71')
axes[1].axvspan(-0.5, 5.5, alpha=0.2, color='red')
axes[1].axhline(y=100/24, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label='Equal Distribution (4.17%)')
axes[1].set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Percentage of Daily Trips (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Share of Daily Trip Volume by Hour', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(0, 24))
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Night Transport Desert Analysis (12 AM - 6 AM)', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/02_temporal/night_transport_desert.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: night_transport_desert.png")

In [0]:
# Summarizing all temporal analysis findings for the final report

temporal_findings = {
    "Monthly Trends": {
        "Highest Volume Month": f"{monthly_stats.loc[monthly_stats['total_trips'].idxmax(), 'month_name']} - {monthly_stats['total_trips'].max():,} trips",
        "Lowest Volume Month": f"{monthly_stats.loc[monthly_stats['total_trips'].idxmin(), 'month_name']} - {monthly_stats['total_trips'].min():,} trips",
        "Month-to-Month Growth": f"{((monthly_stats.iloc[2]['total_trips'] - monthly_stats.iloc[0]['total_trips'])/monthly_stats.iloc[0]['total_trips']*100):.1f}%"
    },
    
    "Weekend vs Weekday": {
        "Weekend Trips": f"{weekend_row['total_trips']:,} ({weekend_row['total_trips']/weekend_comparison['total_trips'].sum()*100:.1f}%)",
        "Weekday Trips": f"{weekday_row['total_trips']:,} ({weekday_row['total_trips']/weekend_comparison['total_trips'].sum()*100:.1f}%)",
        "Weekend Premium": f"${weekend_row['avg_fare'] - weekday_row['avg_fare']:.2f} higher fare",
        "Weekend Tip Premium": f"${weekend_row['avg_tips'] - weekday_row['avg_tips']:.2f} higher tips"
    },
    
    "Night Transport Desert": {
        "Night Volume (12 AM - 6 AM)": f"{total_night_trips:,} trips ({night_pct:.1f}% of daily)",
        "Worst Hour": f"{int(night_data.loc[night_data['trip_count'].idxmin(), 'pickup_hour'])}:00 AM - {int(night_data['trip_count'].min()):,} trips",
        "Average Night Wait": f"{night_data['avg_wait'].mean():.2f} minutes",
        "Policy Impact": "Late-night service significantly reduced - supports need for public transit expansion"
    }
}

print("\n" + "="*80)
print("TEMPORAL ANALYSIS - KEY FINDINGS")
print("="*80)

for category, findings in temporal_findings.items():
    print(f"\n{category.upper()}:")
    for key, value in findings.items():
        print(f"  • {key}: {value}")

print("\n✅ Temporal analysis complete")
print("="*80)

In [0]:
display(df.limit(20))

In [0]:


# Download CSV with pandas
pdf_zones = pd.read_csv(
    "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
)

# Convert to Spark DataFrame
zones = spark.createDataFrame(pdf_zones)

print(f"✅ Zones loaded: {zones.count()} locations")
display(zones)

# Alias DataFrames for join
df_alias = df.alias("trips")
zones_pickup = zones.alias("pickup_zones")
zones_dropoff = zones.alias("dropoff_zones")

# Join zone info to trip data using qualified column names
df_joined = (
    df_alias
    .join(
        zones_pickup.select(
            F.col("pickup_zones.LocationID").alias("pickup_location_id"),
            F.col("pickup_zones.Borough").alias("pickup_borough"),
            F.col("pickup_zones.Zone").alias("pickup_zone")
        ),
        F.col("trips.pulocation_id") == F.col("pickup_location_id"),
        "left"
    )
    .join(
        zones_dropoff.select(
            F.col("dropoff_zones.LocationID").alias("dropoff_location_id"),
            F.col("dropoff_zones.Borough").alias("dropoff_borough"),
            F.col("dropoff_zones.Zone").alias("dropoff_zone")
        ),
        F.col("trips.dolocation_id") == F.col("dropoff_location_id"),
        "left"
    )
)

print("✅ Zone data joined to trips")

df = df_joined

# Checking borough distribution
display(
    df.groupBy("pickup_borough")
      .count()
      .orderBy("count", ascending=False)
)


In [0]:
# Analyzing ride patterns across NYC's five boroughs

print("Analyzing borough-level patterns...")

borough_stats = df.groupBy("pickup_borough").agg(
    F.count("*").alias("total_trips"),
    F.avg("wait_time_minutes").alias("avg_wait"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("trip_miles").alias("avg_distance"),
    F.sum("co2_kg").alias("total_co2_kg")
).orderBy("total_trips", ascending=False).toPandas()

print("\nBorough Statistics:")
print(borough_stats)

# Creating comprehensive borough comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Panel 1: Trip volume by borough
axes[0, 0].barh(borough_stats['pickup_borough'], borough_stats['total_trips'], 
                color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black')
axes[0, 0].set_xlabel('Number of Trips', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Trip Volume by Borough', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_stats['total_trips']):
    axes[0, 0].text(v + 200000, i, f"{v/1e6:.1f}M", va='center', fontweight='bold')

# Panel 2: Average wait time
axes[0, 1].barh(borough_stats['pickup_borough'], borough_stats['avg_wait'],
                color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black')
axes[0, 1].set_xlabel('Wait Time (minutes)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Average Wait Time by Borough', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_stats['avg_wait']):
    axes[0, 1].text(v + 0.1, i, f"{v:.2f}", va='center', fontweight='bold')

# Panel 3: Average fare
axes[1, 0].barh(borough_stats['pickup_borough'], borough_stats['avg_fare'],
                color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black')
axes[1, 0].set_xlabel('Average Fare ($)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Average Fare by Borough', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_stats['avg_fare']):
    axes[1, 0].text(v + 0.5, i, f"${v:.2f}", va='center', fontweight='bold')

# Panel 4: CO2 emissions
axes[1, 1].barh(borough_stats['pickup_borough'], borough_stats['total_co2_kg']/1000,
                color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black')
axes[1, 1].set_xlabel('CO₂ Emissions (metric tons)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Total Emissions by Borough', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_stats['total_co2_kg']/1000):
    axes[1, 1].text(v + 500, i, f"{v/1000:.1f}K", va='center', fontweight='bold')

plt.suptitle('Borough-Level Analysis - Q1 2025', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/03_geographic/borough_comparison.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: borough_comparison.png")

In [0]:
# Identifying best and worst served zones

print("Identifying service quality by zone...")

zone_stats = df.groupBy("pickup_zone", "pickup_borough").agg(
    F.count("*").alias("total_trips"),
    F.avg("wait_time_minutes").alias("avg_wait")
).filter(F.col("pickup_zone").isNotNull())

# Top 20 most served zones
top_zones = zone_stats.orderBy("total_trips", ascending=False).limit(20).toPandas()

# Top 20 longest wait times (service deserts)
worst_wait_zones = zone_stats.filter(F.col("total_trips") > 1000).orderBy("avg_wait", ascending=False).limit(20).toPandas()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Panel 1: Top zones by volume
axes[0].barh(range(len(top_zones)), top_zones['total_trips'], color='steelblue', edgecolor='black')
axes[0].set_yticks(range(len(top_zones)))
axes[0].set_yticklabels([f"{row['pickup_zone'][:30]}" for _, row in top_zones.iterrows()], fontsize=9)
axes[0].set_xlabel('Number of Trips', fontsize=12, fontweight='bold')
axes[0].set_title('Top 20 Most Active Zones', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Panel 2: Worst wait time zones (service deserts)
axes[1].barh(range(len(worst_wait_zones)), worst_wait_zones['avg_wait'], color='#e74c3c', edgecolor='black')
axes[1].set_yticks(range(len(worst_wait_zones)))
axes[1].set_yticklabels([f"{row['pickup_zone'][:30]}" for _, row in worst_wait_zones.iterrows()], fontsize=9)
axes[1].set_xlabel('Average Wait Time (minutes)', fontsize=12, fontweight='bold')
axes[1].set_title('Top 20 Service Desert Zones (Longest Waits)', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Geographic Service Inequality', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/03_geographic/service_inequality_zones.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: service_inequality_zones.png")

In [0]:
# Analyzing shared ride usage and patterns

print("Analyzing shared ride patterns...")

shared_stats = df.groupBy("shared_match_flag").agg(
    F.count("*").alias("total_trips"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("trip_miles").alias("avg_distance"),
    F.avg("driver_pay").alias("avg_driver_pay")
).toPandas()

shared_stats['ride_type'] = shared_stats['shared_match_flag'].map({'Y': 'Shared', 'N': 'Solo'})

print("\nShared vs Solo Rides:")
print(shared_stats)

shared_pct = (shared_stats[shared_stats['shared_match_flag'] == 'Y']['total_trips'].values[0] / 
              shared_stats['total_trips'].sum() * 100)

print(f"\n🚗 SHARED RIDE FINDINGS:")
print(f"  Shared rides: {shared_stats[shared_stats['shared_match_flag'] == 'Y']['total_trips'].values[0]:,} ({shared_pct:.2f}%)")
print(f"  Solo rides: {shared_stats[shared_stats['shared_match_flag'] == 'N']['total_trips'].values[0]:,} ({100-shared_pct:.2f}%)")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Panel 1: Share of rides
labels = shared_stats['ride_type']
sizes = shared_stats['total_trips']
colors = ['#3498db', '#e74c3c']
explode = (0.05, 0)

axes[0].pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=90, textprops={'fontsize': 12, 'weight': 'bold'})
axes[0].set_title('Shared vs Solo Rides', fontsize=14, fontweight='bold')

# Panel 2: Fare comparison
axes[1].bar(shared_stats['ride_type'], shared_stats['avg_fare'], 
            color=colors, edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Average Fare ($)', fontsize=12, fontweight='bold')
axes[1].set_title('Average Fare by Ride Type', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(shared_stats['avg_fare']):
    axes[1].text(i, v + 0.5, f"${v:.2f}", ha='center', fontweight='bold')

# Panel 3: Driver pay comparison
axes[2].bar(shared_stats['ride_type'], shared_stats['avg_driver_pay'],
            color=colors, edgecolor='black', linewidth=1.5)
axes[2].set_ylabel('Average Driver Pay ($)', fontsize=12, fontweight='bold')
axes[2].set_title('Driver Pay by Ride Type', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(shared_stats['avg_driver_pay']):
    axes[2].text(i, v + 0.3, f"${v:.2f}", ha='center', fontweight='bold')

plt.suptitle('Shared Ride Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/06_economic/shared_ride_analysis.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: shared_ride_analysis.png")

In [0]:
# Deep comparison between the two platforms

print("Analyzing Uber vs Lyft...")

company_stats = df.groupBy("company_name").agg(
    F.count("*").alias("total_trips"),
    F.avg("wait_time_minutes").alias("avg_wait"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("tips").alias("avg_tips"),
    F.avg("driver_pay").alias("avg_driver_pay"),
    F.avg("driver_pct").alias("avg_driver_pct"),
    F.avg("trip_miles").alias("avg_distance")
).orderBy("total_trips", ascending=False).toPandas()

print("\nUber vs Lyft Statistics:")
print(company_stats)

# Calculating market share
company_stats['market_share'] = (company_stats['total_trips'] / company_stats['total_trips'].sum() * 100)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Panel 1: Market share
axes[0, 0].pie(company_stats['market_share'], labels=company_stats['company_name'],
               colors=['#000000', '#FF00BF'], autopct='%1.1f%%', startangle=90,
               textprops={'fontsize': 12, 'weight': 'bold'}, explode=(0.05, 0))
axes[0, 0].set_title('Market Share', fontsize=14, fontweight='bold')

# Panel 2: Wait times
axes[0, 1].bar(company_stats['company_name'], company_stats['avg_wait'],
               color=['#000000', '#FF00BF'], edgecolor='black', linewidth=1.5)
axes[0, 1].set_ylabel('Wait Time (minutes)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Average Wait Time', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(company_stats['avg_wait']):
    axes[0, 1].text(i, v + 0.05, f"{v:.2f}", ha='center', fontweight='bold')

# Panel 3: Fares
axes[0, 2].bar(company_stats['company_name'], company_stats['avg_fare'],
               color=['#000000', '#FF00BF'], edgecolor='black', linewidth=1.5)
axes[0, 2].set_ylabel('Fare ($)', fontsize=12, fontweight='bold')
axes[0, 2].set_title('Average Fare', fontsize=14, fontweight='bold')
axes[0, 2].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(company_stats['avg_fare']):
    axes[0, 2].text(i, v + 0.3, f"${v:.2f}", ha='center', fontweight='bold')

# Panel 4: Tips
axes[1, 0].bar(company_stats['company_name'], company_stats['avg_tips'],
               color=['#000000', '#FF00BF'], edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('Tips ($)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Average Tips', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(company_stats['avg_tips']):
    axes[1, 0].text(i, v + 0.02, f"${v:.2f}", ha='center', fontweight='bold')

# Panel 5: Driver pay
axes[1, 1].bar(company_stats['company_name'], company_stats['avg_driver_pay'],
               color=['#000000', '#FF00BF'], edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('Driver Pay ($)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Average Driver Pay', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(company_stats['avg_driver_pay']):
    axes[1, 1].text(i, v + 0.2, f"${v:.2f}", ha='center', fontweight='bold')

# Panel 6: Driver percentage
axes[1, 2].bar(company_stats['company_name'], company_stats['avg_driver_pct'],
               color=['#000000', '#FF00BF'], edgecolor='black', linewidth=1.5)
axes[1, 2].set_ylabel('Driver Take Rate (%)', fontsize=12, fontweight='bold')
axes[1, 2].set_title('Driver Earnings %', fontsize=14, fontweight='bold')
axes[1, 2].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(company_stats['avg_driver_pct']):
    axes[1, 2].text(i, v + 0.5, f"{v:.1f}%", ha='center', fontweight='bold')

plt.suptitle('Uber vs Lyft Platform Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/07_comparison/uber_vs_lyft.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: uber_vs_lyft.png")

In [0]:
# Identifying pollution hotspots at the zone level

print("Analyzing environmental impact by zone...")

zone_emissions = df.groupBy("pickup_zone", "pickup_borough").agg(
    F.sum("co2_kg").alias("total_co2_kg"),
    F.count("*").alias("total_trips"),
    F.avg("trip_miles").alias("avg_distance")
).filter(F.col("pickup_zone").isNotNull())

zone_emissions = zone_emissions.withColumn("co2_per_trip_kg", F.col("total_co2_kg") / F.col("total_trips"))

# Top 20 pollution zones
top_polluters = zone_emissions.orderBy("total_co2_kg", ascending=False).limit(20).toPandas()

# Zones with highest per-trip emissions (inefficiency)
inefficient_zones = zone_emissions.filter(F.col("total_trips") > 1000).orderBy("co2_per_trip_kg", ascending=False).limit(20).toPandas()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Panel 1: Top polluting zones (total emissions)
axes[0].barh(range(len(top_polluters)), top_polluters['total_co2_kg']/1000, color='#e74c3c', edgecolor='black')
axes[0].set_yticks(range(len(top_polluters)))
axes[0].set_yticklabels([f"{row['pickup_zone'][:30]}" for _, row in top_polluters.iterrows()], fontsize=9)
axes[0].set_xlabel('Total CO₂ Emissions (metric tons)', fontsize=12, fontweight='bold')
axes[0].set_title('Top 20 Pollution Hotspots (Total Emissions)', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Panel 2: Most inefficient zones (highest per-trip emissions)
axes[1].barh(range(len(inefficient_zones)), inefficient_zones['co2_per_trip_kg'], color='#f39c12', edgecolor='black')
axes[1].set_yticks(range(len(inefficient_zones)))
axes[1].set_yticklabels([f"{row['pickup_zone'][:30]}" for _, row in inefficient_zones.iterrows()], fontsize=9)
axes[1].set_xlabel('CO₂ per Trip (kg)', fontsize=12, fontweight='bold')
axes[1].set_title('Most Carbon-Inefficient Zones', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Environmental Analysis - Pollution Hotspots', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/05_environmental/pollution_hotspots.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: pollution_hotspots.png")

In [0]:
# Comparing environmental efficiency across boroughs

print("Analyzing borough-level environmental metrics...")

borough_env = df.groupBy("pickup_borough").agg(
    F.sum("co2_kg").alias("total_co2_kg"),
    F.count("*").alias("total_trips"),
    F.avg("co2_kg").alias("avg_co2_per_trip")
).toPandas()

borough_env = borough_env.sort_values("total_co2_kg", ascending=False)

# Calculating carbon intensity (emissions per trip)
borough_env['co2_tons'] = borough_env['total_co2_kg'] / 1000

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Panel 1: Total emissions
axes[0].barh(borough_env['pickup_borough'], borough_env['co2_tons'],
             color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('Total CO₂ Emissions (metric tons)', fontsize=12, fontweight='bold')
axes[0].set_title('Total Carbon Footprint by Borough', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_env['co2_tons']):
    axes[0].text(v + 200, i, f"{v/1000:.1f}K", va='center', fontweight='bold')

# Panel 2: Emissions per trip (efficiency)
axes[1].barh(borough_env['pickup_borough'], borough_env['avg_co2_per_trip'],
             color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'], edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('Average CO₂ per Trip (kg)', fontsize=12, fontweight='bold')
axes[1].set_title('Carbon Efficiency by Borough', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(borough_env['avg_co2_per_trip']):
    axes[1].text(v + 0.02, i, f"{v:.2f}", va='center', fontweight='bold')

plt.suptitle('Borough Environmental Impact Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/05_environmental/borough_emissions.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: borough_emissions.png")

In [0]:
# Creating an interactive hourly demand heatmap by day of week
print("Creating interactive hourly heatmap...")

hourly_daily = df.groupBy("day_name", "pickup_day_of_week", "pickup_hour").agg(
    F.count("*").alias("trip_count")
).orderBy("pickup_day_of_week", "pickup_hour").toPandas()

heatmap_data = hourly_daily.pivot(index='day_name', columns='pickup_hour', values='trip_count')
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(day_order)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=[f"{h}:00" for h in heatmap_data.columns],
    y=heatmap_data.index,
    colorscale='YlOrRd',
    colorbar=dict(title="Trips"),
    hovertemplate='Day: %{y}<br>Hour: %{x}<br>Trips: %{z:,.0f}<extra></extra>'
))

fig.update_layout(
    title='Demand Heatmap: Day of Week vs Hour',
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    font=dict(size=12, family='Arial Black'),
    height=500,
    width=1200
)

fig.write_html('/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/02_temporal/demand_heatmap.html')

fig.show()
print("Saved: demand_heatmap.html (interactive)")

In [0]:
# Creating a flow diagram showing trips between boroughs

print("Creating borough flow visualization...")

borough_flows = df.groupBy("pickup_borough", "dropoff_borough").agg(
    F.count("*").alias("trip_count")
).filter(
    (F.col("pickup_borough").isNotNull()) & 
    (F.col("dropoff_borough").isNotNull())
).orderBy("trip_count", ascending=False).limit(30).toPandas()

# Preparing data for Sankey
boroughs = list(set(borough_flows['pickup_borough'].tolist() + borough_flows['dropoff_borough'].tolist()))
borough_idx = {b: i for i, b in enumerate(boroughs)}

source = [borough_idx[b] for b in borough_flows['pickup_borough']]
target = [borough_idx[b] + len(boroughs) for b in borough_flows['dropoff_borough']]
value = borough_flows['trip_count'].tolist()

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label=boroughs + [f"{b} (Drop)" for b in boroughs],
        color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6'] * 2
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])

fig.update_layout(
    title='Trip Flows Between Boroughs',
    font=dict(size=14, family='Arial Black'),
    height=600,
    width=1200
)

fig.write_html('/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/03_geographic/borough_flows.html')
fig.show()

print("✅ Saved: borough_flows.html (interactive)")

In [0]:
# Analyzing driver pay patterns and inequalities

print("Analyzing driver earnings...")

driver_earnings = df.select("driver_pay", "driver_pct", "company_name", "pickup_borough").sample(fraction=0.01).toPandas()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Panel 1: Driver pay distribution
axes[0, 0].hist(driver_earnings['driver_pay'], bins=50, color='#2ecc71', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Driver Pay ($)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Driver Pay Distribution', fontsize=14, fontweight='bold')
axes[0, 0].axvline(driver_earnings['driver_pay'].median(), color='red', linestyle='--', linewidth=2, label=f"Median: ${driver_earnings['driver_pay'].median():.2f}")
axes[0, 0].set_xlim(0, 60)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Panel 2: Driver percentage distribution
axes[0, 1].hist(driver_earnings['driver_pct'], bins=50, color='#f39c12', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Driver Take Rate (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Driver Earnings Percentage', fontsize=14, fontweight='bold')
axes[0, 1].axvline(driver_earnings['driver_pct'].median(), color='red', linestyle='--', linewidth=2, label=f"Median: {driver_earnings['driver_pct'].median():.1f}%")
axes[0, 1].set_xlim(0, 100)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Panel 3: Driver pay by company
driver_earnings.boxplot(column='driver_pay', by='company_name', ax=axes[1, 0])
axes[1, 0].set_xlabel('Company', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Driver Pay ($)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Driver Pay by Company', fontsize=14, fontweight='bold')
axes[1, 0].get_figure().suptitle('')
axes[1, 0].grid(True, alpha=0.3)

# Panel 4: Driver pay by borough
driver_earnings.boxplot(column='driver_pay', by='pickup_borough', ax=axes[1, 1])
axes[1, 1].set_xlabel('Borough', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Driver Pay ($)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Driver Pay by Borough', fontsize=14, fontweight='bold')
axes[1, 1].get_figure().suptitle('')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Driver Earnings Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()

plt.savefig(
    '/Workspace/Repos/mayowaaloko@gmail.com/fhvhv-trip-data-analysis/visualizations/06_economic/driver_earnings.png',
    dpi=300,
    bbox_inches='tight',
    facecolor='white'
)
display(plt.gcf())
plt.close()

print("✅ Saved: driver_earnings.png")

In [0]:
# Creating the complete findings report

print("\n" + "="*80)
print("PHASE 2: EXPLORATORY DATA ANALYSIS - COMPLETE FINDINGS")
print("="*80)

final_report = {
    "1. Market Overview": {
        "Total Q1 Trips": "57,068,378",
        "Uber Market Share": "73.6% (41.99M trips)",
        "Lyft Market Share": "26.4% (15.08M trips)",
        "Shared Rides": "2.0% (1.14M trips)"
    },
    
    "2. Geographic Patterns": {
        "Top Borough": "Manhattan - 37.5% of trips",
        "Worst Wait Times": "EWR (47.3 min), Staten Island (6.1 min)",
        "Highest Fares": "EWR ($69.55), Manhattan ($26.91)",
        "Service Deserts": "Outer boroughs, especially late night"
    },
    
    "3. Temporal Insights": {
        "Peak Hour": "6 PM (18:00)",
        "Night Desert Hours": "3-5 AM (only 12% of daily trips)",
        "Weekend vs Weekday": "Weekends: longer waits (+7%), lower tips (-10.7%)",
        "Monthly Trend": "January highest (19.4M), slight Q1 decline"
    },
    
    "4. Service Equity": {
        "Manhattan Wait": "4.3 min",
        "EWR Wait": "47.3 min (10x worse)",
        "Accessibility Rate": "9.4% WAV-matched trips",
        "Request Fulfillment": "~100% (low demand, not unmet need)"
    },
    
    "5. Environmental Impact": {
        "Total Q1 Emissions": "102,574 metric tons CO₂",
        "Equivalent To": "22,299 cars driven for 1 year",
        "Top Polluter": "Manhattan (37,800 metric tons)",
        "Per-Trip Average": "1.80 kg CO₂"
    },
    
    "6. Economic Analysis": {
        "Driver Take Rate": "61.7% of total fare",
        "Uber Driver Pay": "$18.08 avg",
        "Lyft Driver Pay": "$17.36 avg",
        "Shared vs Solo Pay": "Shared $18.80 vs Solo $17.87"
    },
    
    "7. Platform Comparison": {
        "Uber Fares": "$24.24 avg (7.5% higher)",
        "Uber Tips": "$0.91 vs Lyft $1.04",
        "Uber Wait": "4.62 min vs Lyft 4.81 min",
        "Driver Earnings": "Similar (~62% take rate both)"
    }
}

for category, findings in final_report.items():
    print(f"\n{category}")
    for key, value in findings.items():
        print(f"  • {key}: {value}")


In [0]:
# Saving the dataset with borough information added

print("Saving updated dataset with geographic data...")

df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("nyc_ridehail_q1_2025_with_geography")

print("✅ Data saved: nyc_ridehail_q1_2025_with_geography")
print(f"Total records: {df.count():,}")
print(f"Total columns: {len(df.columns)}")

In [0]:
# Building a date dimension table for time-based analysis in Power BI

print("Creating date dimension table...")

dim_date = df.select("pickup_date", "pickup_month", "pickup_day_of_week", "day_name", "is_weekend") \
    .distinct() \
    .orderBy("pickup_date")

# Adding additional date attributes
dim_date = dim_date.withColumn("month_name",
    F.when(F.col("pickup_month") == 1, "January")
     .when(F.col("pickup_month") == 2, "February")
     .when(F.col("pickup_month") == 3, "March")
)

dim_date = dim_date.withColumn("week_of_year", F.weekofyear("pickup_date"))
dim_date = dim_date.withColumn("day_of_month", F.dayofmonth("pickup_date"))

# Saving as Delta table
dim_date.write.format("delta").mode("overwrite").saveAsTable("dim_date")

print(f"✅ dim_date created: {dim_date.count()} unique dates")
dim_date.show(10)

In [0]:
# Creating a location lookup table with zone and borough info

print("Creating location dimension table...")

dim_location = zones.select(
    F.col("LocationID").alias("location_id"),
    F.col("Borough").alias("borough"),
    F.col("Zone").alias("zone"),
    F.col("service_zone").alias("service_zone")
).distinct()

# Saving as Delta table
dim_location.write.format("delta").mode("overwrite").saveAsTable("dim_location")

print(f"✅ dim_location created: {dim_location.count()} zones")
dim_location.show(10)

In [0]:
# Simple company lookup table

print("Creating company dimension table...")

dim_company = df.select("hvfhs_license_num", "company_name").distinct()

dim_company.write.format("delta").mode("overwrite").saveAsTable("dim_company")

print(f"✅ dim_company created: {dim_company.count()} companies")
dim_company.show()

In [0]:
# Creating the main fact table with all key metrics for Power BI

print("Creating fact table...")

fact_trips = df.select(
    # Keys
    "pickup_date",
    "pulocation_id",
    "dolocation_id",
    "hvfhs_license_num",
    
    # Time attributes
    "pickup_hour",
    "pickup_day_of_week",
    "is_weekend",
    
    # Trip metrics
    "trip_miles",
    "trip_time",
    "speed_mph",
    
    # Financial metrics
    "base_passenger_fare",
    "tips",
    "tolls",
    "congestion_surcharge",
    "sales_tax",
    "passenger_total_paid",
    "driver_pay",
    "driver_pct",
    "fare_per_mile",
    
    # Environmental
    "co2_kg",
    "co2_lbs",
    
    # Service quality
    "wait_time_minutes",
    
    # Flags
    "shared_match_flag",
    "access_a_ride_flag",
    "wav_match_flag"
)

# Saving as Delta table
fact_trips.write.format("delta").mode("overwrite").saveAsTable("fact_trips")

print(f"✅ fact_trips created: {fact_trips.count():,} records")
print(f"Columns: {len(fact_trips.columns)}")

In [0]:
# Pre-aggregating hourly data for faster Power BI performance

print("Creating hourly aggregated table...")

agg_hourly = df.groupBy("pickup_hour", "company_name").agg(
    F.count("*").alias("total_trips"),
    F.avg("wait_time_minutes").alias("avg_wait_time"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("trip_miles").alias("avg_distance"),
    F.avg("driver_pay").alias("avg_driver_pay"),
    F.sum("co2_kg").alias("total_co2_kg"),
    F.countDistinct("pulocation_id").alias("unique_pickup_zones")
)

agg_hourly.write.format("delta").mode("overwrite").saveAsTable("agg_hourly")

print(f"✅ agg_hourly created: {agg_hourly.count()} records")
agg_hourly.orderBy("pickup_hour").show()

In [0]:
# Pre-aggregating zone-level data

print("Creating zone aggregated table...")

agg_zones = df.groupBy("pulocation_id", "pickup_zone", "pickup_borough").agg(
    F.count("*").alias("total_trips"),
    F.avg("wait_time_minutes").alias("avg_wait_time"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("fare_per_mile").alias("avg_fare_per_mile"),
    F.sum("co2_kg").alias("total_co2_kg"),
    F.avg("driver_pay").alias("avg_driver_pay"),
    (F.sum(F.when(F.col("wav_match_flag") == "Y", 1).otherwise(0)) / F.count("*") * 100).alias("wav_pct")
).filter(F.col("pulocation_id").isNotNull())

agg_zones.write.format("delta").mode("overwrite").saveAsTable("agg_zones")

print(f"✅ agg_zones created: {agg_zones.count()} zones")
agg_zones.orderBy("total_trips", ascending=False).display(10)

In [0]:
# Creating a single-row table with all key metrics for Power BI cards

print("Creating KPI summary table...")

kpi_summary = df.agg(
    F.count("*").alias("total_trips"),
    F.sum("base_passenger_fare").alias("total_revenue"),
    F.sum("driver_pay").alias("total_driver_earnings"),
    F.avg("wait_time_minutes").alias("avg_wait_time"),
    F.avg("base_passenger_fare").alias("avg_fare"),
    F.avg("trip_miles").alias("avg_distance"),
    F.avg("driver_pct").alias("avg_driver_pct"),
    F.sum("co2_kg").alias("total_co2_kg"),
    F.countDistinct("pulocation_id").alias("zones_served"),
    (F.sum(F.when(F.col("wav_match_flag") == "Y", 1).otherwise(0)) / F.count("*") * 100).alias("accessibility_pct"),
    (F.sum(F.when(F.col("shared_match_flag") == "Y", 1).otherwise(0)) / F.count("*") * 100).alias("shared_ride_pct")
)

# Adding calculated metrics
kpi_summary = kpi_summary.withColumn("co2_metric_tons", F.col("total_co2_kg") / 1000)
kpi_summary = kpi_summary.withColumn("car_equivalents", F.col("co2_metric_tons") / 4.6)
kpi_summary = kpi_summary.withColumn("platform_revenue", F.col("total_revenue") - F.col("total_driver_earnings"))

kpi_df = kpi_summary.toPandas()

print("\n📊 KEY PERFORMANCE INDICATORS:")
print(f"  Total Trips: {kpi_df['total_trips'].iloc[0]:,.0f}")
print(f"  Total Revenue: ${kpi_df['total_revenue'].iloc[0]:,.2f}")
print(f"  Driver Earnings: ${kpi_df['total_driver_earnings'].iloc[0]:,.2f}")
print(f"  Platform Revenue: ${kpi_df['platform_revenue'].iloc[0]:,.2f}")
print(f"  Avg Wait: {kpi_df['avg_wait_time'].iloc[0]:.2f} min")
print(f"  CO₂ Emissions: {kpi_df['co2_metric_tons'].iloc[0]:,.0f} metric tons")
print(f"  Accessibility Rate: {kpi_df['accessibility_pct'].iloc[0]:.2f}%")

# Saving as Delta table
spark.createDataFrame(kpi_df).write.format("delta").mode("overwrite").saveAsTable("kpi_summary")

print("\n✅ kpi_summary created")

In [0]:
# Exporting key tables to CSV for Power BI import

print("Exporting tables to CSV...")

export_path = "/Volumes/workspace/default/exports"

# Creating export directory
dbutils.fs.mkdirs(export_path)

# Exporting dimension tables
print("  Exporting dim_date...")
dim_date.toPandas().to_csv(f"{export_path}/dim_date.csv", index=False)

print("  Exporting dim_location...")
dim_location.toPandas().to_csv(f"{export_path}/dim_location.csv", index=False)

print("  Exporting dim_company...")
dim_company.toPandas().to_csv(f"{export_path}/dim_company.csv", index=False)

# Exporting aggregated tables (sampling fact table as it's too large)
print("  Exporting agg_hourly...")
agg_hourly.toPandas().to_csv(f"{export_path}/agg_hourly.csv", index=False)

print("  Exporting agg_zones...")
agg_zones.toPandas().to_csv(f"{export_path}/agg_zones.csv", index=False)

print("  Exporting kpi_summary...")
kpi_df.to_csv(f"{export_path}/kpi_summary.csv", index=False)

# Exporting a sampled fact table (1M records for Power BI testing)
print("  Exporting fact_trips_sample (1M records)...")
fact_trips.sample(fraction=0.02).toPandas().to_csv(f"{export_path}/fact_trips_sample.csv", index=False)

print("\n✅ All tables exported to CSV")
print(f"Location: {export_path}")

In [0]:
# Documenting all tables and columns for reference

print("Creating data dictionary...")

data_dictionary = {
    "dim_date": {
        "description": "Date dimension with temporal attributes",
        "columns": {
            "pickup_date": "Date of pickup (YYYY-MM-DD)",
            "pickup_month": "Month number (1-3)",
            "pickup_day_of_week": "Day of week (1=Sunday, 7=Saturday)",
            "day_name": "Day name (Monday-Sunday)",
            "is_weekend": "Weekend flag (Y/N)",
            "month_name": "Month name (January-March)",
            "week_of_year": "Week number of year",
            "day_of_month": "Day of month (1-31)"
        }
    },
    
    "dim_location": {
        "description": "NYC taxi zone lookup table",
        "columns": {
            "location_id": "Unique zone ID (1-263)",
            "borough": "NYC borough name",
            "zone": "Zone name/neighborhood",
            "service_zone": "Service zone classification"
        }
    },
    
    "dim_company": {
        "description": "Ride-hail company lookup",
        "columns": {
            "hvfhs_license_num": "TLC license number (HV0003=Uber, HV0005=Lyft)",
            "company_name": "Company name (Uber/Lyft)"
        }
    },
    
    "fact_trips": {
        "description": "Main fact table with all trip-level metrics",
        "row_count": "57,068,378 records",
        "columns": {
            "pickup_date": "Date of pickup",
            "pulocation_id": "Pickup zone ID",
            "dolocation_id": "Dropoff zone ID",
            "hvfhs_license_num": "Company license number",
            "pickup_hour": "Hour of pickup (0-23)",
            "trip_miles": "Trip distance in miles",
            "trip_time": "Trip duration in seconds",
            "speed_mph": "Average speed in mph",
            "base_passenger_fare": "Base fare amount",
            "tips": "Tip amount",
            "passenger_total_paid": "Total fare charged",
            "driver_pay": "Amount paid to driver",
            "driver_pct": "Driver earnings as % of total",
            "fare_per_mile": "Fare per mile",
            "co2_kg": "CO₂ emissions in kg",
            "wait_time_minutes": "Wait time from request to pickup",
            "shared_match_flag": "Shared ride indicator (Y/N)",
            "wav_match_flag": "Wheelchair accessible vehicle (Y/N)"
        }
    },
    
    "agg_hourly": {
        "description": "Pre-aggregated hourly metrics by company",
        "columns": {
            "pickup_hour": "Hour of day (0-23)",
            "company_name": "Uber or Lyft",
            "total_trips": "Number of trips",
            "avg_wait_time": "Average wait time (minutes)",
            "avg_fare": "Average fare ($)",
            "total_co2_kg": "Total emissions (kg)"
        }
    },
    
    "agg_zones": {
        "description": "Pre-aggregated zone-level metrics",
        "columns": {
            "pulocation_id": "Zone ID",
            "pickup_zone": "Zone name",
            "pickup_borough": "Borough name",
            "total_trips": "Number of trips",
            "avg_wait_time": "Average wait time (minutes)",
            "avg_fare": "Average fare ($)",
            "total_co2_kg": "Total emissions (kg)",
            "wav_pct": "Wheelchair accessibility rate (%)"
        }
    },
    
    "kpi_summary": {
        "description": "Single-row table with all key metrics for dashboard cards",
        "columns": {
            "total_trips": "Total Q1 trips",
            "total_revenue": "Total revenue ($)",
            "total_driver_earnings": "Total driver pay ($)",
            "avg_wait_time": "Average wait time (min)",
            "co2_metric_tons": "Total emissions (metric tons)",
            "accessibility_pct": "WAV match rate (%)",
            "shared_ride_pct": "Shared ride rate (%)"
        }
    }
}

# Saving data dictionary as JSON
import json

with open(f"{export_path}/data_dictionary.json", "w") as f:
    json.dump(data_dictionary, f, indent=2)

print("✅ Data dictionary created: data_dictionary.json")